<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/02_sam_evaluador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# -*- coding: utf-8 -*-
"""
================================================================================
EVALUADOR SAM 2.0
================================================================================
Sistema de evaluacion para modelos SAM 2.0 (Segment Anything Model 2) con
estructura estandarizada para comparacion de modelos de segmentacion automatica.

CARACTERISTICAS PRINCIPALES:
- Procesamiento incremental con sistema de checkpoint robusto
- Gestion optima de memoria para Google Colab gratuito
- Guardado automatico despues de cada imagen procesada
- 3 configuraciones de generacion optimizadas para fotografia de retrato
- Filtrado inteligente de personas basado en heuristicas del estado del arte
- Visualizaciones 3-panel avanzadas (Original + Todas las mascaras + Personas)
- NPZ completo con todos los scores de SAM 2.0 para analisis posterior
- JSON ligero con metricas de rendimiento (sin analisis geometrico)
- Soporte para 4 modelos SAM 2.0 (Tiny, Small, Base Plus, Large)

CONFIGURACIONES DE GENERACION:
1. low_cost: Ultra-rapida, optimizada para modelos ligeros
2. balanced: Equilibrio calidad-velocidad (recomendado para retratos)
3. quality: Maxima calidad de segmentacion

FILTRADO DE PERSONAS (Heuristicas del estado del arte):
- Ratio de aspecto: 0.3 - 1.5 (personas son mas altas que anchas)
- Area: 5% - 90% de la imagen (personas significativas)
- Confianza SAM: predicted_iou >= 0.85
- Posicion central: Distancia al centro <= 0.3
- Compacidad: Objetos compactos (no fragmentados)
- Sistema de scoring: Minimo 3 de 5 criterios cumplidos

ALMACENAMIENTO NPZ
- segmentation: Mascara binaria
- bbox, area: Geometria basica
- predicted_iou, stability_score: Scores unicos de SAM 2.0
- crop_box: Area de procesamiento usada
- person_confidence: Score de filtrado de personas
- criteria_met, area_ratio, aspect_ratio, center_distance: Metricas de filtrado

VISUALIZACIONES 3-PANEL:
- Panel 1: Imagen original
- Panel 2: TODAS las mascaras generadas por SAM (overlay con colores)
- Panel 3: SOLO personas filtradas (bounding boxes verdes + scores)

ESTRUCTURA DE SALIDA:
/TFM/2_Modelos/sam2/{modelo}/{config}/
├── resultados_{timestamp}.json     # Metricas y metadatos ligeros
├── mascaras/                        # Mascaras NPZ completas por imagen
└── visualizaciones/                 # PNGs 3-panel con informacion

Entrada: Imagenes desde /TFM/0_Imagenes/
Salida: JSON, mascaras NPZ completas y visualizaciones en estructura organizada

Autor: Jesús L.
Proyecto: TFM - Evaluacion Comparativa de Tecnicas de Segmentacion
Universidad: Universidad Oberta de Cataluna (UOC)
Fecha: Octubre 2025
================================================================================
"""

'\n================================================================================\nEVALUADOR SAM 2.0\n================================================================================\nSistema de evaluacion para modelos SAM 2.0 (Segment Anything Model 2) con\nestructura estandarizada para comparacion de modelos de segmentacion automatica.\n\nCARACTERISTICAS PRINCIPALES:\n- Procesamiento incremental con sistema de checkpoint robusto\n- Gestion optima de memoria para Google Colab gratuito\n- Guardado automatico despues de cada imagen procesada\n- 3 configuraciones de generacion optimizadas para fotografia de retrato\n- Filtrado inteligente de personas basado en heuristicas del estado del arte\n- Visualizaciones 3-panel avanzadas (Original + Todas las mascaras + Personas)\n- NPZ completo con todos los scores de SAM 2.0 para analisis posterior\n- JSON ligero con metricas de rendimiento (sin analisis geometrico)\n- Soporte para 4 modelos SAM 2.0 (Tiny, Small, Base Plus, Large)\n\nCONFIGURA

In [1]:
# =============================================================================
# SETUP GOOGLE COLAB Y DESCARGA DE CHECKPOINTS
# =============================================================================

import os
from pathlib import Path

# Montar Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    IN_COLAB = True
    print("Google Drive montado correctamente")
except:
    IN_COLAB = False
    print("No estamos en Colab")

# IMPORTANTE: Checkpoints dentro del directorio sam2
CHECKPOINTS_DIR = Path("/content/drive/MyDrive/TFM/2_Modelos/sam2/checkpoints")
CHECKPOINTS_DIR.mkdir(parents=True, exist_ok=True)

# URLs oficiales de Meta AI
CHECKPOINTS = {
    'sam2_hiera_tiny.pt': 'https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt',
    'sam2_hiera_small.pt': 'https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt',
    'sam2_hiera_base_plus.pt': 'https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_base_plus.pt',
    'sam2_hiera_large.pt': 'https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt',
}

print("="*80)
print("DESCARGANDO CHECKPOINTS SAM 2.0")
print("="*80)

for filename, url in CHECKPOINTS.items():
    checkpoint_path = CHECKPOINTS_DIR / filename

    if checkpoint_path.exists():
        size_mb = checkpoint_path.stat().st_size / (1024 * 1024)
        print(f"[EXISTE] {filename}: {size_mb:.1f} MB")
    else:
        print(f"[DESCARGANDO] {filename}...")
        !wget -q --show-progress {url} -O {checkpoint_path}

        if checkpoint_path.exists():
            size_mb = checkpoint_path.stat().st_size / (1024 * 1024)
            print(f"  Completado: {size_mb:.1f} MB")
        else:
            print(f"  ERROR descargando {filename}")

print("\n" + "="*80)
print(f"Checkpoints listos en: {CHECKPOINTS_DIR}")
print("="*80)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado correctamente
DESCARGANDO CHECKPOINTS SAM 2.0
[DESCARGANDO] sam2_hiera_tiny.pt...
/content/drive/MyDr 100%[===================>] 148.68M  44.8MB/s    in 3.3s    
  Completado: 148.7 MB
[DESCARGANDO] sam2_hiera_small.pt...
/content/drive/MyDr 100%[===================>] 175.77M  48.3MB/s    in 4.2s    
  Completado: 175.8 MB
[DESCARGANDO] sam2_hiera_base_plus.pt...
/content/drive/MyDr 100%[===================>] 308.51M  58.0MB/s    in 5.2s    
  Completado: 308.5 MB
[DESCARGANDO] sam2_hiera_large.pt...
/content/drive/MyDr 100%[===================>] 856.35M  63.4MB/s    in 16s     
  Completado: 856.4 MB

Checkpoints listos en: /content/drive/MyDrive/TFM/2_Modelos/sam2/checkpoints


In [2]:
# =============================================================================
# INSTALACION DE DEPENDENCIAS
# =============================================================================

print("Instalando dependencias...")

# Dependencias core
!pip install -q torch torchvision
!pip install -q opencv-python matplotlib Pillow
!pip install -q numpy scipy

# SAM 2.0
!pip install -q git+https://github.com/facebookresearch/segment-anything-2.git

print("Dependencias instaladas correctamente")

Instalando dependencias...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Dependencias instaladas correctamente


In [3]:
# =============================================================================
# IMPORTACIONES
# =============================================================================

import os
import sys
import json
import hashlib
import warnings
import gc
import time
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Optional, Tuple

import numpy as np
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# SAM 2.0
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

warnings.filterwarnings('ignore')

# Configurar dispositivo
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if DEVICE == 'cuda':
    print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
    print(f"VRAM total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("Usando CPU (sera mas lento)")

print("Librerias importadas correctamente")

GPU disponible: Tesla T4
VRAM total: 14.74 GB
Librerias importadas correctamente


In [4]:
# =============================================================================
# CONFIGURACION DE MODELOS Y PARAMETROS
# =============================================================================

@dataclass
class ModeloSAMInfo:
    """Informacion completa de un modelo SAM 2.0"""
    nombre: str
    checkpoint: str
    config: str
    parametros_millones: int
    vram_estimada_gb: float
    descripcion: str

    def obtener_nombre_sanitizado(self) -> str:
        """Obtiene nombre corto para archivos"""
        return self.nombre.replace('sam2_hiera_', '')

# Catalogo de modelos SAM 2.0 disponibles
MODELOS_DISPONIBLES = {
    'tiny': ModeloSAMInfo(
        nombre='sam2_hiera_tiny',
        checkpoint='sam2_hiera_tiny.pt',
        config='sam2_hiera_t.yaml',
        parametros_millones=39,
        vram_estimada_gb=2.0,
        descripcion='Modelo ultraligero optimizado para edge computing'
    ),
    'small': ModeloSAMInfo(
        nombre='sam2_hiera_small',
        checkpoint='sam2_hiera_small.pt',
        config='sam2_hiera_s.yaml',
        parametros_millones=46,
        vram_estimada_gb=3.0,
        descripcion='Balance optimo calidad-velocidad para produccion'
    ),
    'base_plus': ModeloSAMInfo(
        nombre='sam2_hiera_base_plus',
        checkpoint='sam2_hiera_base_plus.pt',
        config='sam2_hiera_b+.yaml',
        parametros_millones=80,
        vram_estimada_gb=5.0,
        descripcion='Modelo avanzado para produccion de alta calidad'
    ),
    'large': ModeloSAMInfo(
        nombre='sam2_hiera_large',
        checkpoint='sam2_hiera_large.pt',
        config='sam2_hiera_l.yaml',
        parametros_millones=224,
        vram_estimada_gb=8.0,
        descripcion='Modelo de investigacion - maxima calidad'
    )
}

@dataclass
class ConfiguracionSAM:
    """
    Configuracion de parametros para generacion automatica de mascaras.

    Parametros basados en:
    - Meta AI SAM 2.0 documentation (2024)
    - Best practices para fotografia de retrato
    - Optimizacion para Google Colab gratuito

    Referencias:
    - Kirillov et al. (2024) "Segment Anything 2"
    - Ravi et al. (2024) "SAM 2: Segment Anything in Images and Videos"
    """
    nombre: str
    descripcion: str

    # Parametros de generacion de puntos
    points_per_side: int
    points_per_batch: int

    # Umbrales de calidad
    pred_iou_thresh: float
    stability_score_thresh: float
    stability_score_offset: float

    # Control de mascaras y crops
    crop_n_layers: int
    crop_n_points_downscale_factor: int
    min_mask_region_area: int

# Configuraciones de generacion optimizadas
CONFIGURACIONES_GENERACION = {
    'low_cost': ConfiguracionSAM(
        nombre='low_cost',
        descripcion='Configuracion ultra-rapida optimizada para Tiny',
        points_per_side=16,          # 16x16 = 256 prompts (rapido)
        points_per_batch=64,
        pred_iou_thresh=0.88,
        stability_score_thresh=0.92,
        stability_score_offset=1.0,
        crop_n_layers=0,             # Sin crops para ahorrar memoria
        crop_n_points_downscale_factor=1,
        min_mask_region_area=200     # Filtrar mascaras muy pequeñas
    ),
    'balanced': ConfiguracionSAM(
        nombre='balanced',
        descripcion='Configuracion equilibrada - Recomendada para retratos',
        points_per_side=24,          # 24x24 = 576 prompts (balance)
        points_per_batch=64,
        pred_iou_thresh=0.86,
        stability_score_thresh=0.92,
        stability_score_offset=1.0,
        crop_n_layers=1,
        crop_n_points_downscale_factor=2,
        min_mask_region_area=150
    ),
    'quality': ConfiguracionSAM(
        nombre='quality',
        descripcion='Maxima calidad - Mayor precision en bordes',
        points_per_side=32,          # 32x32 = 1024 prompts (completo)
        points_per_batch=64,
        pred_iou_thresh=0.86,
        stability_score_thresh=0.92,
        stability_score_offset=1.0,
        crop_n_layers=1,
        crop_n_points_downscale_factor=2,
        min_mask_region_area=100
    )
}

print("Configuraciones de modelos y parametros cargadas")

Configuraciones de modelos y parametros cargadas


In [5]:
# =============================================================================
# UTILIDADES
# =============================================================================

class Utilidades:
    """Funciones auxiliares para procesamiento de imagenes y gestion de memoria"""

    @staticmethod
    def liberar_memoria():
        """Libera memoria GPU y RAM"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    @staticmethod
    def cargar_imagen(ruta: Path, max_size: int = 1024) -> np.ndarray:
        """
        Carga y redimensiona imagen manteniendo aspect ratio.

        Args:
            ruta: Path a la imagen
            max_size: Dimension maxima permitida

        Returns:
            Imagen como array numpy RGB
        """
        try:
            imagen = cv2.imread(str(ruta))
            if imagen is None:
                raise ValueError(f"No se pudo cargar la imagen: {ruta}")

            # Convertir BGR a RGB
            imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

            # Redimensionar si es necesario
            h, w = imagen.shape[:2]
            if max(h, w) > max_size:
                escala = max_size / max(h, w)
                nuevo_w = int(w * escala)
                nuevo_h = int(h * escala)
                imagen = cv2.resize(imagen, (nuevo_w, nuevo_h),
                                  interpolation=cv2.INTER_AREA)

            return imagen

        except Exception as e:
            raise ValueError(f"Error cargando imagen {ruta}: {str(e)}")

    @staticmethod
    def calcular_hash_imagen(ruta: Path) -> str:
        """Calcula hash MD5 para identificacion unica de imagen"""
        try:
            with open(ruta, 'rb') as f:
                return hashlib.md5(f.read()).hexdigest()[:12]
        except Exception:
            return "hash_error"

    @staticmethod
    def guardar_json(datos: Any, archivo: Path, indent: int = 2):
        """Guarda datos en formato JSON con conversion de tipos numpy"""
        def convertir_tipos(obj):
            if isinstance(obj, np.integer):
                return int(obj)
            elif isinstance(obj, np.floating):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, dict):
                return {k: convertir_tipos(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [convertir_tipos(item) for item in obj]
            return obj

        try:
            archivo.parent.mkdir(parents=True, exist_ok=True)
            datos_convertidos = convertir_tipos(datos)

            with open(archivo, 'w', encoding='utf-8') as f:
                json.dump(datos_convertidos, f, indent=indent,
                         ensure_ascii=False, default=str)
        except Exception as e:
            raise IOError(f"Error guardando JSON en {archivo}: {str(e)}")

print("Clase de utilidades definida")

Clase de utilidades definida


In [6]:
# =============================================================================
# GESTOR DE CHECKPOINT (COMO ONEFORMER)
# =============================================================================

class GestorCheckpoint:
    """
    Gestor de checkpoint para tracking de imagenes procesadas.
    Similar a OneFormer: solo almacena nombres de imagenes completadas.
    """

    def __init__(self, ruta_checkpoint: Path):
        """
        Args:
            ruta_checkpoint: Path al archivo checkpoint.json
        """
        self.ruta_checkpoint = ruta_checkpoint
        self.completadas = self._cargar_checkpoint()

    def _cargar_checkpoint(self) -> set:
        """Carga conjunto de imagenes completadas"""
        if self.ruta_checkpoint.exists():
            try:
                with open(self.ruta_checkpoint, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    return set(data.get('completadas', []))
            except Exception as e:
                print(f"Advertencia: Error cargando checkpoint: {str(e)}")
                return set()
        return set()

    def esta_completada(self, nombre_imagen: str) -> bool:
        """Verifica si una imagen ya fue procesada"""
        return nombre_imagen in self.completadas

    def marcar_completada(self, nombre_imagen: str):
        """Marca una imagen como completada y guarda checkpoint"""
        self.completadas.add(nombre_imagen)
        self._guardar_checkpoint()

    def _guardar_checkpoint(self):
        """Guarda checkpoint en disco"""
        try:
            self.ruta_checkpoint.parent.mkdir(parents=True, exist_ok=True)
            data = {
                'completadas': sorted(list(self.completadas)),
                'total': len(self.completadas),
                'ultima_actualizacion': datetime.now().isoformat()
            }
            with open(self.ruta_checkpoint, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)
        except Exception as e:
            print(f"Error guardando checkpoint: {str(e)}")

    def obtener_estadisticas(self) -> Dict[str, Any]:
        """Retorna estadisticas del checkpoint"""
        return {
            'total_completadas': len(self.completadas),
            'imagenes': sorted(list(self.completadas))
        }

print("Clase gestor de checkpoint definida")

Clase gestor de checkpoint definida


In [7]:
# =============================================================================
# FILTRADO DE PERSONAS CON HEURISTICAS DEL ESTADO DEL ARTE
# =============================================================================

class FiltradorPersonas:
    """
    Filtrado de mascaras para detectar personas en fotografia de retrato.

    Implementa heuristicas basadas en investigacion del estado del arte:

    Referencias:
    - Lin et al. (2014) "Microsoft COCO: Common Objects in Context"
    - Kirillov et al. (2023) "Segment Anything"
    - Dollar et al. (2012) "Pedestrian Detection: An Evaluation"

    Criterios de filtrado (minimo 3 de 5):
    1. Area: 5%-90% de imagen
    2. Aspect ratio: 0.3-1.5
    3. Confianza SAM: predicted_iou >= 0.85
    4. Posicion central: Distancia <= 0.3
    5. Compacidad: >0.2
    """

    def __init__(self):
        self.MIN_AREA_RATIO = 0.05
        self.MAX_AREA_RATIO = 0.90
        self.MIN_ASPECT_RATIO = 0.3
        self.MAX_ASPECT_RATIO = 1.5
        self.MIN_PREDICTED_IOU = 0.85
        self.PREFERRED_CENTER_DISTANCE = 0.3
        self.MIN_COMPACTNESS = 0.2

    def filtrar_mascaras_personas(self,
                                  mascaras: List[Dict],
                                  imagen_shape: Tuple[int, int]) -> List[Dict]:
        """Filtra mascaras que probablemente son personas"""
        if not mascaras:
            return []

        h, w = imagen_shape
        area_total = h * w
        centro_x, centro_y = w / 2, h / 2

        mascaras_filtradas = []

        for mask_data in mascaras:
            segmentacion = mask_data['segmentation']
            bbox = mask_data['bbox']
            predicted_iou = mask_data.get('predicted_iou', 0)

            area = np.sum(segmentacion)
            area_ratio = area / area_total

            x, y, mask_w, mask_h = bbox
            aspect_ratio = mask_w / mask_h if mask_h > 0 else 0

            mask_centro_x = x + mask_w / 2
            mask_centro_y = y + mask_h / 2

            dist_centro = np.sqrt(
                ((mask_centro_x - centro_x) / w) ** 2 +
                ((mask_centro_y - centro_y) / h) ** 2
            )

            criterios_cumplidos = 0

            if self.MIN_AREA_RATIO <= area_ratio <= self.MAX_AREA_RATIO:
                criterios_cumplidos += 1

            if self.MIN_ASPECT_RATIO <= aspect_ratio <= self.MAX_ASPECT_RATIO:
                criterios_cumplidos += 1

            if predicted_iou >= self.MIN_PREDICTED_IOU:
                criterios_cumplidos += 1

            if dist_centro <= self.PREFERRED_CENTER_DISTANCE:
                criterios_cumplidos += 1

            perimetro_aprox = 2 * (mask_w + mask_h)
            compacidad = (4 * np.pi * area) / (perimetro_aprox ** 2) if perimetro_aprox > 0 else 0
            if compacidad > self.MIN_COMPACTNESS:
                criterios_cumplidos += 1

            confidence_score = criterios_cumplidos / 5.0

            if criterios_cumplidos >= 3:
                mask_data['person_confidence'] = confidence_score
                mask_data['criteria_met'] = criterios_cumplidos
                mask_data['area_ratio'] = float(area_ratio)
                mask_data['aspect_ratio'] = float(aspect_ratio)
                mask_data['center_distance'] = float(dist_centro)
                mascaras_filtradas.append(mask_data)

        mascaras_filtradas.sort(key=lambda x: x['person_confidence'], reverse=True)

        return mascaras_filtradas

print("Clase de filtrado de personas definida")

Clase de filtrado de personas definida


In [8]:
# =============================================================================
# GESTOR DE MASCARAS - UN NPZ POR IMAGEN CON TODAS LAS MASCARAS
# =============================================================================

class GestorMascaras:
    """
    Gestiona el almacenamiento de mascaras en formato NPZ.
    Estructura como OneFormer: UN archivo NPZ por imagen conteniendo TODAS las mascaras.
    """

    def __init__(self, directorio_mascaras: Path):
        self.directorio_mascaras = directorio_mascaras
        self.directorio_mascaras.mkdir(parents=True, exist_ok=True)

    def guardar_mascaras(self,
                        mascaras_personas: List[Dict],
                        nombre_archivo: str) -> Dict[str, Any]:
        """
        Guarda TODAS las mascaras de una imagen en UN solo archivo NPZ.

        Args:
            mascaras_personas: Lista de mascaras filtradas
            nombre_archivo: Nombre del archivo de imagen

        Returns:
            Metadatos para el JSON
        """
        try:
            nombre_base = Path(nombre_archivo).stem
            archivo_npz = self.directorio_mascaras / f"{nombre_base}.npz"

            if not mascaras_personas:
                # Guardar NPZ vacio
                np.savez_compressed(
                    archivo_npz,
                    num_mascaras=np.array(0, dtype=np.int32)
                )
                return {
                    'archivo_mascaras': archivo_npz.name,
                    'num_mascaras': 0,
                    'mascaras': []
                }

            # Preparar arrays consolidados
            num_mascaras = len(mascaras_personas)

            # Arrays de mascaras binarias (stack de todas las mascaras)
            primera_mascara = mascaras_personas[0]['segmentation']
            h, w = primera_mascara.shape

            mascaras_binarias = np.zeros((num_mascaras, h, w), dtype=np.uint8)
            bboxes = np.zeros((num_mascaras, 4), dtype=np.float32)
            areas = np.zeros(num_mascaras, dtype=np.int32)
            predicted_ious = np.zeros(num_mascaras, dtype=np.float32)
            stability_scores = np.zeros(num_mascaras, dtype=np.float32)
            person_confidences = np.zeros(num_mascaras, dtype=np.float32)
            criteria_mets = np.zeros(num_mascaras, dtype=np.int32)
            area_ratios = np.zeros(num_mascaras, dtype=np.float32)
            aspect_ratios = np.zeros(num_mascaras, dtype=np.float32)
            center_distances = np.zeros(num_mascaras, dtype=np.float32)

            # Llenar arrays
            for idx, mask_data in enumerate(mascaras_personas):
                mascaras_binarias[idx] = mask_data['segmentation'].astype(np.uint8)
                bboxes[idx] = np.array(mask_data['bbox'], dtype=np.float32)
                areas[idx] = mask_data['area']
                predicted_ious[idx] = mask_data.get('predicted_iou', 0.0)
                stability_scores[idx] = mask_data.get('stability_score', 0.0)
                person_confidences[idx] = mask_data.get('person_confidence', 0.0)
                criteria_mets[idx] = mask_data.get('criteria_met', 0)
                area_ratios[idx] = mask_data.get('area_ratio', 0.0)
                aspect_ratios[idx] = mask_data.get('aspect_ratio', 0.0)
                center_distances[idx] = mask_data.get('center_distance', 0.0)

            # Guardar TODO en un solo NPZ
            np.savez_compressed(
                archivo_npz,
                num_mascaras=np.array(num_mascaras, dtype=np.int32),
                mascaras_binarias=mascaras_binarias,
                bboxes=bboxes,
                areas=areas,
                predicted_ious=predicted_ious,
                stability_scores=stability_scores,
                person_confidences=person_confidences,
                criteria_mets=criteria_mets,
                area_ratios=area_ratios,
                aspect_ratios=aspect_ratios,
                center_distances=center_distances
            )

            # Metadatos para JSON (resumen por mascara)
            mascaras_info = []
            for idx in range(num_mascaras):
                mascaras_info.append({
                    'indice': int(idx),
                    'area': int(areas[idx]),
                    'bbox': [float(x) for x in bboxes[idx]],
                    'predicted_iou': float(predicted_ious[idx]),
                    'stability_score': float(stability_scores[idx]),
                    'person_confidence': float(person_confidences[idx]),
                    'criteria_met': int(criteria_mets[idx])
                })

            return {
                'archivo_mascaras': archivo_npz.name,
                'num_mascaras': num_mascaras,
                'mascaras': mascaras_info
            }

        except Exception as e:
            print(f"Error guardando mascaras: {str(e)}")
            return {
                'archivo_mascaras': None,
                'num_mascaras': 0,
                'mascaras': [],
                'error': str(e)
            }

print("Clase gestor de mascaras definida")

Clase gestor de mascaras definida


In [9]:
# =============================================================================
# GENERADOR DE VISUALIZACIONES 3-PANEL
# =============================================================================

class GeneradorVisualizacionesSAM:
    """Genera visualizaciones 3-panel de resultados SAM 2.0"""

    def __init__(self, directorio_visualizaciones: Path):
        self.directorio_viz = directorio_visualizaciones
        self.directorio_viz.mkdir(parents=True, exist_ok=True)

    def generar_visualizacion(self,
                             imagen_original: np.ndarray,
                             nombre_archivo: str,
                             resultado_generacion: Dict,
                             titulo_modelo: str) -> Optional[str]:
        """Genera visualizacion 3-panel y la guarda"""
        try:
            mascaras_todas = resultado_generacion.get('mascaras_todas', [])
            mascaras_personas = resultado_generacion.get('mascaras_personas', [])

            if not mascaras_todas:
                return None

            fig, axes = plt.subplots(1, 3, figsize=(20, 7))
            fig.suptitle(
                f'{titulo_modelo} - {nombre_archivo}\n'
                f'Total: {len(mascaras_todas)} mascaras | Personas: {len(mascaras_personas)}',
                fontsize=16, fontweight='bold'
            )

            # Panel 1: Original
            axes[0].imshow(imagen_original)
            axes[0].set_title('Imagen Original', fontsize=14)
            axes[0].axis('off')

            # Panel 2: Todas las mascaras
            self._visualizar_mascaras(
                axes[1], imagen_original, mascaras_todas,
                f'Todas las Mascaras ({len(mascaras_todas)})'
            )

            # Panel 3: Solo personas
            self._visualizar_mascaras(
                axes[2], imagen_original, mascaras_personas,
                f'Personas Detectadas ({len(mascaras_personas)})',
                mostrar_confianza=True
            )

            # Guardar
            nombre_base = Path(nombre_archivo).stem
            ruta_salida = self.directorio_viz / f"{nombre_base}.png"
            plt.tight_layout()
            plt.savefig(ruta_salida, dpi=150, bbox_inches='tight')
            plt.close(fig)

            return ruta_salida.name

        except Exception as e:
            print(f"Error generando visualizacion: {str(e)}")
            plt.close('all')
            return None

    def _visualizar_mascaras(self, ax, imagen, mascaras, titulo, mostrar_confianza=False):
        """Visualiza mascaras en un eje"""
        ax.imshow(imagen)

        if not mascaras:
            ax.set_title(f'{titulo} - Vacio', fontsize=12)
            ax.axis('off')
            return

        overlay = np.zeros_like(imagen, dtype=np.float32)

        for mask_data in mascaras:
            seg = mask_data['segmentation']
            color = np.random.random(3)

            for c in range(3):
                overlay[:, :, c] += seg * color[c] * 0.6

        overlay = np.clip(overlay, 0, 1)
        ax.imshow(overlay, alpha=0.5)

        if mostrar_confianza:
            for mask_data in mascaras:
                bbox = mask_data['bbox']
                confianza = mask_data.get('person_confidence', 0)

                rect = mpatches.Rectangle(
                    (bbox[0], bbox[1]), bbox[2], bbox[3],
                    fill=False, edgecolor='lime', linewidth=2
                )
                ax.add_patch(rect)

                ax.text(
                    bbox[0], bbox[1] - 5,
                    f'{confianza:.2f}',
                    color='lime', fontsize=10, fontweight='bold',
                    bbox=dict(facecolor='black', alpha=0.7, pad=2)
                )

        ax.set_title(titulo, fontsize=12)
        ax.axis('off')

print("Clase generador de visualizaciones definida")

Clase generador de visualizaciones definida


In [10]:
# =============================================================================
# GENERADOR DE MASCARAS SAM 2.0
# =============================================================================

class GeneradorMascarasSAM:
    """Genera mascaras automaticas usando SAM 2.0"""

    def __init__(self,
                 modelo_info: ModeloSAMInfo,
                 config_sam: ConfiguracionSAM,
                 checkpoints_path: Path):
        self.modelo_info = modelo_info
        self.config_sam = config_sam
        self.checkpoints_path = checkpoints_path

        self.modelo = None
        self.mask_generator = None
        self.filtrador = FiltradorPersonas()

    def cargar_modelo(self):
        """Carga el modelo SAM 2.0"""
        try:
            print(f"Cargando modelo {self.modelo_info.nombre}...")

            checkpoint_path = self.checkpoints_path / self.modelo_info.checkpoint

            if not checkpoint_path.exists():
                raise FileNotFoundError(
                    f"Checkpoint no encontrado: {checkpoint_path}"
                )

            device = "cuda" if torch.cuda.is_available() else "cpu"
            self.modelo = build_sam2(
                config_file=self.modelo_info.config,
                ckpt_path=str(checkpoint_path),
                device=device
            )

            self.mask_generator = SAM2AutomaticMaskGenerator(
                model=self.modelo,
                points_per_side=self.config_sam.points_per_side,
                points_per_batch=self.config_sam.points_per_batch,
                pred_iou_thresh=self.config_sam.pred_iou_thresh,
                stability_score_thresh=self.config_sam.stability_score_thresh,
                stability_score_offset=self.config_sam.stability_score_offset,
                crop_n_layers=self.config_sam.crop_n_layers,
                crop_n_points_downscale_factor=self.config_sam.crop_n_points_downscale_factor,
                min_mask_region_area=self.config_sam.min_mask_region_area,
            )

            print(f"Modelo cargado correctamente")

        except Exception as e:
            print(f"Error cargando modelo: {str(e)}")
            raise

    def generar_mascaras(self, imagen: np.ndarray) -> Dict[str, Any]:
        """Genera mascaras automaticas y filtra personas"""
        try:
            inicio = time.time()

            mascaras_todas = self.mask_generator.generate(imagen)

            tiempo_generacion = (time.time() - inicio) * 1000

            mascaras_personas = self.filtrador.filtrar_mascaras_personas(
                mascaras_todas,
                imagen.shape[:2]
            )

            resultado = {
                'total_mascaras_generadas': len(mascaras_todas),
                'personas_detectadas': len(mascaras_personas),
                'mascaras_personas': mascaras_personas,
                'mascaras_todas': mascaras_todas,
                'tiempo_generacion_ms': tiempo_generacion,
                'imagen_shape': imagen.shape[:2],
            }

            return resultado

        except Exception as e:
            print(f"Error generando mascaras: {str(e)}")
            raise

    def liberar_memoria(self):
        """Libera memoria del modelo"""
        del self.modelo
        del self.mask_generator
        self.modelo = None
        self.mask_generator = None
        Utilidades.liberar_memoria()

print("Clase generador de mascaras definida")

Clase generador de mascaras definida


In [11]:
# =============================================================================
# PROCESADOR DE IMAGENES
# =============================================================================

class ProcesadorImagenes:
    """Procesador principal que coordina la generacion, guardado y visualizacion"""

    def __init__(self,
                 modelo_info: ModeloSAMInfo,
                 config_sam: ConfiguracionSAM,
                 checkpoints_path: Path,
                 directorio_salida: Path,
                 max_size: int = 1024,
                 generar_visualizaciones: bool = True):
        self.modelo_info = modelo_info
        self.config_sam = config_sam
        self.max_size = max_size
        self.generar_vis = generar_visualizaciones

        # Crear estructura de directorios
        self.dir_mascaras = directorio_salida / "mascaras"
        self.dir_json = directorio_salida / "json"
        self.dir_viz = directorio_salida / "visualizaciones"

        self.dir_mascaras.mkdir(parents=True, exist_ok=True)
        self.dir_json.mkdir(parents=True, exist_ok=True)
        if generar_visualizaciones:
            self.dir_viz.mkdir(parents=True, exist_ok=True)

        # Componentes
        self.generador = GeneradorMascarasSAM(
            modelo_info, config_sam, checkpoints_path
        )
        self.gestor_mascaras = GestorMascaras(self.dir_mascaras)

        if generar_visualizaciones:
            self.visualizador = GeneradorVisualizacionesSAM(self.dir_viz)
        else:
            self.visualizador = None

    def inicializar(self):
        """Inicializa el generador cargando el modelo"""
        self.generador.cargar_modelo()

    def procesar_imagen(self, ruta_imagen: Path) -> Dict[str, Any]:
        """Procesa una imagen completa"""
        nombre_archivo = ruta_imagen.name
        nombre_base = ruta_imagen.stem

        try:
            # Cargar imagen
            tiempo_inicio = time.time()
            imagen = Utilidades.cargar_imagen(ruta_imagen, self.max_size)
            hash_imagen = Utilidades.calcular_hash_imagen(ruta_imagen)
            tiempo_carga = (time.time() - tiempo_inicio) * 1000

            # Generar mascaras
            resultado_generacion = self.generador.generar_mascaras(imagen)

            # Guardar mascaras en NPZ (UN archivo con todas las mascaras)
            metadatos_mascaras = self.gestor_mascaras.guardar_mascaras(
                resultado_generacion['mascaras_personas'],
                nombre_archivo
            )

            # Generar visualizacion
            archivo_viz = None
            if self.visualizador and resultado_generacion['personas_detectadas'] > 0:
                modelo_nombre = self.modelo_info.obtener_nombre_sanitizado().upper()
                titulo = f"{modelo_nombre}-{self.config_sam.nombre}"
                archivo_viz = self.visualizador.generar_visualizacion(
                    imagen,
                    nombre_archivo,
                    resultado_generacion,
                    titulo
                )

            # Construir JSON individual para esta imagen
            tiempo_total = tiempo_carga + resultado_generacion['tiempo_generacion_ms']

            resultado = {
                'timestamp': datetime.now().isoformat(),
                'nombre_imagen': nombre_archivo,
                'modelo': self.modelo_info.obtener_nombre_sanitizado(),
                'dataset': 'SAM2',
                'configuracion': self.config_sam.nombre,
                'arquitectura': self.modelo_info.nombre,
                'hash': hash_imagen,
                'dimensiones': {
                    'alto': int(imagen.shape[0]),
                    'ancho': int(imagen.shape[1])
                },
                'resultados': {
                    'personas_detectadas': resultado_generacion['personas_detectadas'],
                    'mascaras_totales': resultado_generacion['total_mascaras_generadas'],
                    'mascaras': metadatos_mascaras
                },
                'procesamiento': {
                    'tiempo_carga_ms': round(tiempo_carga, 2),
                    'tiempo_generacion_ms': round(resultado_generacion['tiempo_generacion_ms'], 2),
                    'tiempo_total_ms': round(tiempo_total, 2)
                },
                'archivos': {
                    'visualizacion': archivo_viz
                }
            }

            # Guardar JSON individual para esta imagen
            ruta_json = self.dir_json / f"{nombre_base}.json"
            Utilidades.guardar_json(resultado, ruta_json)

            return resultado

        except Exception as e:
            print(f"ERROR procesando {nombre_archivo}: {str(e)}")
            import traceback
            traceback.print_exc()

            # JSON de error
            resultado_error = {
                'timestamp': datetime.now().isoformat(),
                'nombre_imagen': nombre_archivo,
                'modelo': self.modelo_info.obtener_nombre_sanitizado(),
                'configuracion': self.config_sam.nombre,
                'error': str(e),
                'exito': False
            }

            ruta_json = self.dir_json / f"{nombre_base}.json"
            Utilidades.guardar_json(resultado_error, ruta_json)

            return resultado_error

    def liberar_recursos(self):
        """Libera recursos del procesador"""
        self.generador.liberar_memoria()

print("Clase procesador de imagenes definida")

Clase procesador de imagenes definida


In [12]:
# =============================================================================
# EVALUADOR PRINCIPAL SAM 2.0
# =============================================================================

class EvaluadorSAM2:
    """Evaluador principal que coordina todo el proceso de evaluacion"""

    def __init__(self,
                 ruta_imagenes: Path,
                 ruta_salida_base: Path,
                 ruta_checkpoints: Path,
                 modelos_evaluar: List[str],
                 configs_evaluar: List[str],
                 max_size: int = 1024,
                 generar_visualizaciones: bool = True,
                 pausa_entre_imagenes: float = 2.0):
        self.ruta_imagenes = ruta_imagenes
        self.ruta_salida_base = ruta_salida_base
        self.ruta_checkpoints = ruta_checkpoints
        self.modelos_evaluar = modelos_evaluar
        self.configs_evaluar = configs_evaluar
        self.max_size = max_size
        self.generar_vis = generar_visualizaciones
        self.pausa = pausa_entre_imagenes

    def evaluar_combinacion(self, modelo_key: str, config_key: str):
        """Evalua una combinacion modelo + configuracion"""
        modelo_info = MODELOS_DISPONIBLES[modelo_key]
        config_sam = CONFIGURACIONES_GENERACION[config_key]

        modelo_nombre = modelo_info.obtener_nombre_sanitizado()
        config_nombre = config_sam.nombre

        # Nombre de directorio: modelo_config (como OneFormer)
        config_id = f"{modelo_nombre}_{config_nombre}"
        dir_salida = self.ruta_salida_base / config_id

        print(f"\n{'='*80}")
        print(f"EVALUANDO: {config_id}")
        print(f"{'='*80}")

        # Checkpoint
        checkpoint_path = dir_salida / "checkpoint.json"
        checkpoint = GestorCheckpoint(checkpoint_path)

        print(f"Imagenes ya completadas: {checkpoint.obtener_estadisticas()['total_completadas']}")

        # Obtener imagenes
        extensiones = ['*.jpg', '*.JPG', '*.png', '*.PNG', '*.jpeg', '*.JPEG']
        imagenes = []
        for ext in extensiones:
            imagenes.extend(self.ruta_imagenes.glob(ext))
        imagenes = sorted(set(imagenes))

        # Filtrar pendientes
        imagenes_pendientes = [
            img for img in imagenes
            if not checkpoint.esta_completada(img.name)
        ]

        print(f"Total imagenes: {len(imagenes)}")
        print(f"Pendientes: {len(imagenes_pendientes)}")

        if not imagenes_pendientes:
            print("Todas las imagenes ya fueron procesadas")
            return

        # Inicializar procesador
        print(f"\nInicializando procesador...")
        procesador = ProcesadorImagenes(
            modelo_info=modelo_info,
            config_sam=config_sam,
            checkpoints_path=self.ruta_checkpoints,
            directorio_salida=dir_salida,
            max_size=self.max_size,
            generar_visualizaciones=self.generar_vis
        )

        procesador.inicializar()

        print(f"\nProcesando {len(imagenes_pendientes)} imagenes...")

        # Procesar imagenes
        for idx, imagen_path in enumerate(imagenes_pendientes, 1):
            print(f"\n[{idx}/{len(imagenes_pendientes)}] {imagen_path.name}")

            try:
                resultado = procesador.procesar_imagen(imagen_path)

                if resultado.get('exito', True):
                    print(f"  Personas: {resultado['resultados']['personas_detectadas']}")
                    print(f"  Tiempo: {resultado['procesamiento']['tiempo_total_ms']:.1f} ms")

                # Marcar como completada
                checkpoint.marcar_completada(imagen_path.name)
                print(f"  Checkpoint actualizado")

            except Exception as e:
                print(f"  ERROR: {str(e)}")
                continue

            finally:
                Utilidades.liberar_memoria()
                if idx < len(imagenes_pendientes):
                    time.sleep(self.pausa)

        # Limpiar
        print(f"\nLiberando recursos...")
        procesador.liberar_recursos()
        del procesador
        Utilidades.liberar_memoria()

        print(f"\n{'='*80}")
        print(f"COMBINACION COMPLETADA: {config_id}")
        print(f"{'='*80}")

    def ejecutar_evaluacion_completa(self):
        """Ejecuta evaluacion completa de todas las combinaciones"""
        print(f"\n{'#'*80}")
        print("INICIO DE EVALUACION SAM 2.0")
        print(f"{'#'*80}")
        print(f"\nConfiguracion:")
        print(f"  Modelos: {self.modelos_evaluar}")
        print(f"  Configuraciones: {self.configs_evaluar}")
        print(f"  Imagenes: {self.ruta_imagenes}")
        print(f"  Salida: {self.ruta_salida_base}")

        total_combinaciones = len(self.modelos_evaluar) * len(self.configs_evaluar)
        combinacion_actual = 0

        for modelo_key in self.modelos_evaluar:
            for config_key in self.configs_evaluar:
                combinacion_actual += 1

                print(f"\n{'#'*80}")
                print(f"COMBINACION {combinacion_actual}/{total_combinaciones}")
                print(f"{'#'*80}")

                try:
                    self.evaluar_combinacion(modelo_key, config_key)

                except Exception as e:
                    print(f"\nERROR FATAL: {str(e)}")
                    import traceback
                    traceback.print_exc()
                    print(f"\nContinuando con siguiente combinacion...")
                    continue

                finally:
                    Utilidades.liberar_memoria()
                    time.sleep(5)

        print(f"\n{'#'*80}")
        print("EVALUACION COMPLETA FINALIZADA")
        print(f"{'#'*80}")

print("Clase evaluador principal definida")

Clase evaluador principal definida


In [13]:
# =============================================================================
# FUNCION MAIN PARA EJECUCION
# =============================================================================

def main():
    """Funcion principal para ejecutar la evaluacion"""

    print("\n" + "="*80)
    print("CONFIGURACION DEL EVALUADOR SAM 2.0")
    print("="*80)

    # CONFIGURACION DE RUTAS
    ruta_imagenes = Path("/content/drive/MyDrive/TFM/0_Imagenes")
    ruta_salida_base = Path("/content/drive/MyDrive/TFM/2_Modelos/sam2")
    ruta_checkpoints = Path("/content/drive/MyDrive/TFM/2_Modelos/sam2/checkpoints")

    # CONFIGURACION DE EVALUACION
    # Empezar con prueba rapida
    modelos_evaluar = ['tiny', 'small', 'base_plus', 'large']
    configs_evaluar = ['low_cost', 'balanced', 'quality']

    # Parametros
    max_size = 1024
    generar_visualizaciones = True
    pausa_entre_imagenes = 2.0

    # Validar configuracion
    print("\nValidando configuracion...")
    try:
        if not ruta_imagenes.exists():
            raise FileNotFoundError(f"Imagenes no encontradas: {ruta_imagenes}")

        if not ruta_checkpoints.exists():
            raise FileNotFoundError(f"Checkpoints no encontrados: {ruta_checkpoints}")

        for modelo_key in modelos_evaluar:
            modelo_info = MODELOS_DISPONIBLES[modelo_key]
            checkpoint_path = ruta_checkpoints / modelo_info.checkpoint
            if not checkpoint_path.exists():
                raise FileNotFoundError(f"Checkpoint no encontrado: {checkpoint_path}")

        print("Configuracion validada")

    except Exception as e:
        print(f"ERROR: {str(e)}")
        return

    # Resumen
    print(f"\nRESUMEN:")
    print(f"  Modelos: {modelos_evaluar}")
    print(f"  Configuraciones: {configs_evaluar}")
    print(f"  Combinaciones: {len(modelos_evaluar) * len(configs_evaluar)}")
    print(f"  Dataset: {ruta_imagenes}")
    print(f"  Salida: {ruta_salida_base}")
    print(f"  Checkpoints: {ruta_checkpoints}")

    # Crear evaluador
    evaluador = EvaluadorSAM2(
        ruta_imagenes=ruta_imagenes,
        ruta_salida_base=ruta_salida_base,
        ruta_checkpoints=ruta_checkpoints,
        modelos_evaluar=modelos_evaluar,
        configs_evaluar=configs_evaluar,
        max_size=max_size,
        generar_visualizaciones=generar_visualizaciones,
        pausa_entre_imagenes=pausa_entre_imagenes
    )

    # Ejecutar
    print(f"\nINICIANDO EVALUACION...")
    evaluador.ejecutar_evaluacion_completa()

    print(f"\n{'='*80}")
    print("EVALUACION FINALIZADA")
    print(f"{'='*80}")

print("Funcion main definida")
print("\n" + "="*80)
print("LISTO PARA EJECUTAR")
print("="*80)
print("\nPara iniciar la evaluacion, ejecuta:")
print(">>> main()")

Funcion main definida

LISTO PARA EJECUTAR

Para iniciar la evaluacion, ejecuta:
>>> main()


In [14]:
if __name__ == "__main__":
    main()


CONFIGURACION DEL EVALUADOR SAM 2.0

Validando configuracion...
Configuracion validada

RESUMEN:
  Modelos: ['tiny', 'small', 'base_plus', 'large']
  Configuraciones: ['low_cost', 'balanced', 'quality']
  Combinaciones: 12
  Dataset: /content/drive/MyDrive/TFM/0_Imagenes
  Salida: /content/drive/MyDrive/TFM/2_Modelos/sam2
  Checkpoints: /content/drive/MyDrive/TFM/2_Modelos/sam2/checkpoints

INICIANDO EVALUACION...

################################################################################
INICIO DE EVALUACION SAM 2.0
################################################################################

Configuracion:
  Modelos: ['tiny', 'small', 'base_plus', 'large']
  Configuraciones: ['low_cost', 'balanced', 'quality']
  Imagenes: /content/drive/MyDrive/TFM/0_Imagenes
  Salida: /content/drive/MyDrive/TFM/2_Modelos/sam2

################################################################################
COMBINACION 1/12
##################################################################